(clas)=
# Clasificación de eventos
Para comparar los algoritmos de las LHCO 2020 utilizamos como base algunos algoritmos sencillos ya implementados en librerías como `scikit-learn`{cite}`scikit-learn` y uno programado usando `tensorflow`{cite}`tensorflow2015-whitepaper`. 

En esta sección se observarán algunas de las característica de la clasificación con estos algoritmos. Los modelos utilizados son:

```{table} Algoritmos utilizados para comparación
:name: clas-alg

|      Nombre                      | Implementación | Tipo                                             |
|:--------------------------------:|:--------------:|:------------------------------------------------:|
|Random Forest Classifier (RFC)    | `scikit-learn` | [Bosque aleatorio](alg-bosques)                  |
|Gradient Boosting Classifier (GBC)| `scikit-learn` | [Clasificador del gradiente del impulso](alg-gbc)|
|Quadratic Discriminant Analysis (QDA) | `scikit-learn` | [Análisis de discriminante cuadrático](alg-qda)  | 
|MLP Classifier                    | `scikit-learn` | [Red neuronal](alg-neural)                       |
|Tensorflow Classifier             | `tensorflow`   | [Red neuronal](alg-neural)                       |
| KMeans                           | `scikit-learn` | [K-means](alg-kmeans)                            |
```
explicados en la {numref}`alg`.

Para este análisis, se utilizó el conjunto de datos R&D.
(clas-importancia)=
## Importancia de las características
Por la forma en la que clasifican, algunos modelos permiten conocer cuáles de las características de los eventos fueron las variables más relevantes para la clasificación. De los modelos en la {numref}`clas-alg`, RFC y GBC asignan puntajes a las variables de acuerdo a su importancia.

A continuación, realizaremos el entrenamiento de los modelos y la clasificación de los datos de prueba. Las celdas siguientes preparan los datos, entrenan los modelos y realizan la clasificación utilizando funciones de `benchtools`.

In [1]:
# Importamos las librerías principales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import argparse
import os.path

# De scikit-learn importamos herramientas
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
# Y los clasificadores
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neural_network import MLPClassifier
from sklearn.cluster import KMeans

# Lo necesario para construir el modelo de tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

# Funciones de benchtools
from benchtools.src.plotools import pred_test_hist, image_grid
from benchtools.src.clustering import build_features
from benchtools.src.datatools import separate_data
from benchtools.scripts.run import TensorflowClassifier, training, evaluate
from benchtools.src.metrictools import rejection_plot, precision_recall_plot

# Definimos semillas para la reproducibilidad
tf.random.set_seed(125)


In [2]:
# Datos a utilizar
path_data = "../../../datos/events_anomalydetection.h5"

In [3]:
# Esta celda se corre una vez para pre-procesar los datos
# Una vez que el archivo existe no vuelve a correr
build_features(path_data=path_data, nbatch=11, outname='RnD-1100000', outdir='../../../datos/', chunksize=100000)

A file with that name already exists


In [4]:
# En esta celda preparamos los datos para utilizar los algoritmos
# Separamos los datos en conjuntos de entrenamiento y prueba
df_RnD = pd.read_csv('../../../datos/RnD-1100000.csv')
X, y = separate_data(df_RnD, standarize=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y) # 70% training y 30% test

# Guardamos las columnas de masa
X_train_m = X_train.loc[:,['m_j1', 'm_j2', 'm_jj']].copy()
X_test_m = X_test.loc[:,['m_j1', 'm_j2', 'm_jj']].copy()

# Eliminamos las masas de los conjuntos de entrenamiento y prueba
X_train.drop(['m_j1', 'm_j2', 'm_jj'], axis=1, inplace=True)
X_test.drop(['m_j1', 'm_j2', 'm_jj'], axis=1, inplace=True)

In [5]:
# Listamos los clasificadores a utilizar
# En conjunto con el scaler
classifiers = [(MinMaxScaler(feature_range=(-1,1)), TensorflowClassifier(input_shape = [X_train.shape[1]])),
                (StandardScaler(), RandomForestClassifier(random_state=1)),
                (RobustScaler(), GradientBoostingClassifier(random_state=4)),
                (RobustScaler(), QuadraticDiscriminantAnalysis()), 
                (StandardScaler(), MLPClassifier(random_state=7)),
                (StandardScaler(), KMeans(n_clusters=2, random_state=15))
                ]

In [6]:
# Con esta función entrenamos los modelos
# Solo hace falta correrla una vez
training(X_train, X_test, y_train, y_test, classifiers, '../../../datos', 'log')


  0%|          | 0/6 [00:00<?, ?it/s]

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 14)               56        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 512)               7680      
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2

100%|██████████| 6/6 [1:38:58<00:00, 989.73s/it]   


Models saved


In [7]:
# En esta celda cargamos los modelos entrenados
models = []

# Cargamos el algoritmo entrenado de tensorflow
tf_model = load_model(os.path.join('../../../datos','tf_model_{}.h5'.format('log')))
models.append(('TensorflowClassifier', tf_model))

# Cargamos los algoritmos entrenados de scikit-learn
with open(os.path.join('../../../datos',"sklearn_models_{}.pckl".format('log')), "rb") as f:
    while True:
        try:
            models.append(pickle.load(f))
        except EOFError:
            break

# Evaluamos
clfs = evaluate(X_test, y_test, models ,train=True)

100%|██████████| 6/6 [01:41<00:00, 16.91s/it]


De los modelos entrenados, se puede obtener la importancia de cada variable. Un gráfico de estos puntajes se observa en la {numref}`clas-feature-imp` para RFC y GBC.

In [8]:
# En esta celda graficamos las características más importantes para los modelos

# Obtenemos las características más importantes
# models[x][y] : x = 1(random forest), 2(gradient boosting); y = 0(nombre del modelo), 1(modelo entrenado) 
fi_rf = models[1][1].steps[1][1].feature_importances_.tolist()
fi_gb = models[2][1].steps[1][1].feature_importances_.tolist()

# Redondeamos los puntajes
weights = [fi_rf, fi_gb]
weights = [[ round(elem, 3) for elem in weight ] for weight in weights]
# Obtenemos los nombres de los modelos
names_fi = [models[1][0], models[2][0]]
# Obtenemos los nombres de las características
features = X_train.columns.tolist()

# Juntamos los puntajes con las características en tuplas
importance = {}
ii = 0
for weight in weights:
    f_i = list(zip(features, weight))
    importance[names_fi[ii]] = f_i
    ii +=1

# Graficamos en un bucle 
colors=['darkorange', 'green']
lista_images = []
ii = 0
for name, scores in importance.items():

    # Ordenamos de menor a mayor
    scores.sort(key=lambda x: x[1], reverse=False) 

    # Salvamos los nombres y su puntaje separados
    # y revertimos las tuplas para tener de mayor a menor puntaje 
    features = list(zip(*scores))[0]
    score = list(zip(*scores))[1]
    x_pos = np.arange(len(features)) 

    # Graficamos
    fig = plt.figure(facecolor='white')
    plt.barh(x_pos, score,align='center', color = colors[ii])
    plt.yticks(x_pos, features) 
    plt.xlabel('Feature importance')
    plt.title('Feature importance: {}'.format(name))
    filename = '../../figuras/{}-feature-importance.png'.format(name)
    lista_images.append(filename)
    plt.savefig(filename, bbox_inches='tight')
    plt.close('all')
    ii += 1
    
image_grid(rows=1, columns=2, images=lista_images, name='clas-feature-imp', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-feature-imp.png
---
name: clas-feature-imp
width: 80%
---
Importancia de las variables utilizadas en el entrenamiento según RFC (izquierda) y GBC (derecha).
```

Notamos que las características más relevantes para ambos clasificadores son la variable de subestructura $\tau_{21}$ y el $p_T$ de los jets. En general, son relevantes las características que presentan distribuciones diferentes para señal y fondo.
(clas-reconstruccion)=
## Reconstrucción de distribuciones
De la clasificación con los distintos modelos, se pueden reconstruir las variables para observar cómo están clasificando los algoritmos. En la {numref}`clas-variables-dist` observamos las distribuciones reales y la clasificación de las variables más importante según {numref}`clas-feature-imp`.

In [9]:
# En esta celda graficamos algunas variables importantes
list_images =[]
variables=['pT_j2', 'tau_21_j1', 'n_hadrons']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X_test.reset_index(drop=True), X_test_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        fig = plt.figure(facecolor='white')
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
    del X_plot, pred, label

image_grid(rows=6, columns=3, images=list_images, name='clas-variables-dist', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-variables-dist.png
---
name: clas-variables-dist
width: 100%
---
Distribución de algunas de las variables más importantes para la clasificación. Cada fila de imágenes representa un clasificador. De arriba a abajo: Tensorflow Classifier, RFC, GBC, QDA, MLP y KMeans. Por columna, de izquierda a derecha, se encuentras las distribuciones de $p_T$ del jet secundario, $\tau_{21}$ del jet principal y el número de hadrones de los eventos.
```
En {numref}`clas-variables-dist` vemos que los modelos, en general, predicen el fondo con bastante precisión. La distinción entre señal y fondo, es lograda en gran medida por los modelos supervisados. Particularmente, vemos que en todas las variables el clasificador de Tensorflow está sobreestimando el fondo y subestimando la señal. Al contrario, RFC, GBC y QDA sobreestiman la señal y subestiman el fondo. El clasificador MLP parece estimar las clases con mayor precisión. 

El único modelo no-supervisado, Kmeans, según los gráficos de $p_T$ y de $\tau_{12}$, logra separar algunos eventos de señal, pero es evidente que en la clasificación de señal hay gran cantidad de fondo. En el plot de número de hadrones vemos que no logra diferencia entre señal y fondo.

Al graficar algunas de las variables con menor importancia según la {numref}`clas-feature-imp`, vemos en la {numref}`clas-variables-noimp-dist`, que los modelos supervisados logran separar señal y fondo. El modelo de TensorFlow parece ser el más preciso estimando $\eta$ del jet principal, variable para la cual el clasificador MLP logra separar señal y fondo pero la distribución de la señal predicha está corrida a la izquierda. 

Nuevamente, Kmeans parece separar siferenciar ligeramente entre señal y fondo en algunas variables. En la variable $\eta$ vemos que no logra separar señal y fondo.


In [10]:
# En esta celda graficamos las variables menos relevantes para la clasificación
list_images =[]
variables=['deltaR_j12', 'E_j1', 'eta_j1']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X_test.reset_index(drop=True), X_test_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        fig = plt.figure(facecolor='white')
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
    del X_plot, pred, label

image_grid(rows=6, columns=3, images=list_images, name='clas-variables-noimp-dist', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-variables-noimp-dist.png
---
name: clas-variables-noimp-dist
width: 100%
---
Distribución de algunas de las variables menos relevantes para la clasificación. Cada fila de imágenes representa un clasificador. De arriba a abajo: Tensorflow Classifier, RFC, GBC, QDA, MLP y KMeans. Por columna, de izquierda a derecha, se encuentras las distribuciones de $\Delta R$, $E$ del jet principal y $\eta$ del jet principal.
```
En la {numref}`clas-masa-dist` podemos ver la predicción de la variable de masa invariante.

In [11]:
# En esta celda graficamos la masa invariante predicha
list_mass_images=[]
variables=['m_jj']
for model in clfs:
    for variable in variables:
        # Obtenemos predicciones como pandas Series
        pred = pd.Series(model.pred.flatten(), name='ypred')
        # Obtenemos las etiquetas como pandas Serie
        label = pd.Series(model.label, name='ytest').reset_index(drop=True)
        # Juntamos las masas, las predicciones y las etiquetas
        X_plot = pd.concat([X_test.reset_index(drop=True), X_test_m.reset_index(drop=True), pred, label], axis=1)
        # Graficamos
        pred_test_hist(X_plot, variable, ypred='ypred', ytest='ytest', n_bins=50, log=False)
        plt.title('{}: distribución de {}'.format(model.name, variable))
        # Guardando el path de cada imagen
        filename = os.path.join('../../figuras/','{}-{}.png'.format(model.name,variable))
        list_mass_images.append(filename)
        # Salvamos la imagen como png
        plt.savefig(filename, bbox_inches='tight', facecolor=fig.get_facecolor(),edgecolor='none')
        plt.close('all')
    del X_plot, pred, label
    
image_grid(rows=2, columns=3, images=list_mass_images, name='clas-masa-dist', path='../../figuras/', remove=True)

```{figure} ./../../figuras/clas-masa-dist.png
---
name: clas-masa-dist
width: 100%
---
Distribución la masa invariante. En la fila superior, de izquierda a derecha, las predicciones de: Tensorflow Classifier, RFC, GBC y en la fila inferior: QDA, MLP y KMeans.
```
Las variables de masa no fueron utilizadas para entrenamiento, pero los modelos supervisados logran obtener distribuciones cercanas, lo que indica que están realizando correctamente la clasificación. El modelo de TensorFlow parece estar subestimando la señal. Los demás modelos supervisados parecen sobreestimar la señal de acuerdo con los gráficos de predicción de $m_{jj}$. Kmeans obtiene un pico para la señal correctamente clasificado, pero es evidente que está clasificando algunos eventos de fondo como señal.

Aunque esta comparación de distribuciones proporciona un punto de partida para la comparación de los algoritmos, es necesario el uso de métricas, porque no es evidente qué algoritmo está clasificando mejor el conjunto de datos.